# Install

In [ ]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3.0.2
!pip install torch

In [ ]:
#깃허브에서 KoBERT 파일 로드
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

# 필요 라이브러리

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

#kobert
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [ ]:
#GPU 사용
device = torch.device("cuda:0")

#BERT 모델, Vocabulary 불러오기
bertmodel, vocab = get_pytorch_kobert_model()

/content/.cache/kobert_v1.zip[██████████████████████████████████████████████████]
/content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece[██████████████████████████████████████████████████]


# 데이터셋 불러오기

In [80]:
import pandas as pd
emotion_data = pd.read_excel('/content/drive/MyDrive/CapstonDesign/한국어 감정대화데이터셋.xlsx')

# 데이터 전처리

In [85]:
emotion_data.sample(n=10)

emotion_data.loc[(emotion_data['Emotion'] == "공포"), 'Emotion'] = 0  #공포 => 0
emotion_data.loc[(emotion_data['Emotion'] == "놀람"), 'Emotion'] = 1  #놀람 => 1
emotion_data.loc[(emotion_data['Emotion'] == "분노"), 'Emotion'] = 2  #분노 => 2
emotion_data.loc[(emotion_data['Emotion'] == "슬픔"), 'Emotion'] = 3  #슬픔 => 3
emotion_data.loc[(emotion_data['Emotion'] == "행복"), 'Emotion'] = 4  #행복 => 4

data_list = []
for q, label in zip(emotion_data['Sentence'], emotion_data['Emotion'])  :
    data = []
    data.append(q)
    data.append(str(label))

    data_list.append(data)

print(data_list[0])
print(data_list[6000])
print(data_list[12000])
print(data_list[18000])
print(data_list[24000])
print(data_list[-1])

['언니 동생으로 부르는게 맞는 일인가요..??', '0']
['기술적으로도 아직도 해체해서 다시 완벽히 돌려놓는게 어려운데 해체를한다고?', '1']
['당연히 그렇게 해야지 우리나라도 판매를 중단하라', '2']
['그거들은 뒤부터 미치겠어요...', '3']
['대박한 앨범인 것 같아요ㅠㅠ', '4']
['유재석 오라버니 해피투게더 봤어요', '4']


# Train data & Test data

In [86]:
#train & test 데이터로 나누기
from sklearn.model_selection import train_test_split

dataset_train, dataset_test = train_test_split(data_list, test_size=0.25, random_state=0)
print(len(dataset_train))
print(len(dataset_test))

21251
7084


# KoBERT 입력 데이터로 만들기

In [83]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

# Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 10
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

# 토큰화와 패딩이 잘 이루어져있는지 확인
data_train[0]

# torch 형식의 dataset
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py:478: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


# KoBERT 학습모델 만들기

In [87]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=5,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

#BERT 모델 불러오기
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
    
train_dataloader

# 모델 저장 및 불러오기

In [91]:
# Load model
model = torch.load('/content/drive/MyDrive/CapstonDesign/model.pt')

# 콘텐츠 기반 필터링(ALBUM 150)
- 컬럼 정보
    - Ranking : 음악 랭킹
    - Music Name : 음악 명
    - Artist Name : 작곡가
    - Genres : 장르
    - Average Rating : 평균 순위
    - Number of Ratings : 조회수
    - Number of Reviews : 음악 리뷰수

In [92]:
fear = "Sweet but Psycho"
surprise = "Moning Mood"
anger = "Centuries"
sadness = "Alone"
happy = "Power Up"

In [93]:
#### - 데이터 읽기
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

music = pd.read_excel('/content/drive/MyDrive/CapstonDesign/MusicDataset/Album_1000_dataset.xlsx')
music_df = music[['Ranking', 'Music Name', 'Artist Name', 'Genres', 'Average Rating', 'Number of Ratings', 'Number of Reviews']]
music_df.head(2)

# genres_literal CountVectorize 수행
count_vect = CountVectorizer(min_df=0, ngram_range=(1, 2))
genre_mat = count_vect.fit_transform(music_df['Genres'])

genre_sim = cosine_similarity(genre_mat, genre_mat)

# 내림차순 정렬을 위해 -1 옵션을 추가로 준다
genre_sim_sorted_ind = genre_sim.argsort()[:, ::-1]


# 추천 영화 DataFrame 반환 함수
def find_sim_music(df, sorted_ind, title_name, top_n=30):
    # 특정 영화 정보 뽑아냄
    title_music = df[df['Music Name']==title_name]
    title_index = title_music.index.values
    similar_indexes = sorted_ind[title_index, :(top_n)].reshape(-1)

    return df.iloc[similar_indexes]


def fear_music_recommend():
    similar_music = find_sim_music(music_df, genre_sim_sorted_ind, fear, 30)
    return similar_music.sample(n=5)[['Music Name', 'Artist Name']]
    


def surprise_music_recommend():
    similar_music = find_sim_music(music_df, genre_sim_sorted_ind, surprise, 30)
    return similar_music.sample(n=5)[['Music Name', 'Artist Name']]


def anger_music_recommend():
    similar_music = find_sim_music(music_df, genre_sim_sorted_ind, anger, 30)
    return similar_music.sample(n=5)[['Music Name', 'Artist Name']]


def sadness_music_recommend():
    similar_music = find_sim_music(music_df, genre_sim_sorted_ind, sadness, 30)
    return similar_music.sample(n=5)[['Music Name', 'Artist Name']]


def happy_music_recommend():
    similar_music = find_sim_music(music_df, genre_sim_sorted_ind, happy, 30)
    return similar_music.sample(n=5)[['Music Name', 'Artist Name']]

# 새로운 문장 테스트

In [94]:
 #토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)


def predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)
    
    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)


        test_eval=[]
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()

            if np.argmax(logits) == 0:
                test_eval.append("공포가")
                print(">> 오늘의 문장에서 " + test_eval[0] + " 느껴집니다.", "\n")
                print(fear_music_recommend())

            elif np.argmax(logits) == 1:
                test_eval.append("놀람이")
                print(">> 오늘의 문장에서 " + test_eval[0] + " 느껴집니다.", "\n")
                print(surprise_music_recommend())
                
            elif np.argmax(logits) == 2:
                test_eval.append("분노가")
                print(">> 오늘의 문장에서 " + test_eval[0] + " 느껴집니다.", "\n")
                print(anger_music_recommend())

            elif np.argmax(logits) == 3:
                test_eval.append("슬픔이")
                print(">> 오늘의 문장에서 " + test_eval[0] + " 느껴집니다.", "\n")
                print(sadness_music_recommend())

            elif np.argmax(logits) == 4:
                test_eval.append("행복이")
                print(">> 오늘의 문장에서 " + test_eval[0] + " 느껴집니다.", "\n")
                print(happy_music_recommend())           

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [95]:
end = 1
while end == 1:
    sentence = input("감정분석을 위해 문장을 입력해주세요 : ")
    if sentence == '0':
        break
    predict(sentence)
    print("\n")

감정분석을 위해 문장을 입력해주세요 : 기분 좋아
>> 오늘의 문장에서 행복이 느껴집니다. 

     Music Name        Artist Name
138  Next Level                에스파
147       붉은 노을            BIGBANG
125  내가 제일 잘 나가               2NE1
129         빠빠빠  크레용 팝(Crayon Pop)
133  After LIKE                아이브


감정분석을 위해 문장을 입력해주세요 : 자연어처리 성공


/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py:478: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


>> 오늘의 문장에서 놀람이 느껴집니다. 

               Music Name Artist Name
46                    밤편지         아이유
39  Four Seasons 'Winter'   Piazzolla
37     Piano Sonata No.21      Mozart
47                     안녕          폴킴
38              Fur Elise   Beethoven


감정분석을 위해 문장을 입력해주세요 : 추천시스템도 끝


/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py:478: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


>> 오늘의 문장에서 놀람이 느껴집니다. 

                Music Name Artist Name
34  Eine Kleine Nachtmusik      Mozart
59                  카페에 앉아     원 모어 찬스
53                      꽃길         김세정
54                  그대라는 시          태연
30             Moning Mood       Grieg


감정분석을 위해 문장을 입력해주세요 : 다했어!!


/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py:478: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


>> 오늘의 문장에서 행복이 느껴집니다. 

    Music Name   Artist Name
143     Psycho          레드벨벳
135  LOVE DIVE           아이브
147      붉은 노을       BIGBANG
128       나팔바지       싸이(PSY)
144   긴 생머리 그녀  틴탑 (TEENTOP)


감정분석을 위해 문장을 입력해주세요 : 내일 발표만 잘하자...


/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py:478: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


>> 오늘의 문장에서 공포가 느껴집니다. 

          Music Name          Artist Name
29    Electric Shock                 f(x)
0   Sweet but Psycho              Ava Max
20          Lollipop  빅뱅 (Bigbang) ＆ 2NE1
23                TT                TWICE
2           Rockabye         Clean Bandit


감정분석을 위해 문장을 입력해주세요 : 0


# 참고문헌

- https://velog.io/@seolini43/KOBERT%EB%A1%9C-%EB%8B%A4%EC%A4%91-%EB%B6%84%EB%A5%98-%EB%AA%A8%EB%8D%B8-%EB%A7%8C%EB%93%A4%EA%B8%B0-%ED%8C%8C%EC%9D%B4%EC%8D%ACColab 
- https://hoit1302.tistory.com/159
- https://hipster4020.tistory.com/109
- 김경재, 「BERT 기반 감성분석을 이용한 추천시스템」, 동국대학교 일반논문, 2021.03
- https://sig413.tistory.com/5
